In [1]:
from spark_batch.lib.spark_session import get_spark_session
from spark_batch.lib.elt_manager import EltManager
from spark_batch.lib.pxlogger import CustomLogger
from spark_batch.lib.util import toTargetCondition

#CustomLogger.set_custom_log_level(logging.DEBUG)  #default log_level is 'INFO'
spark = get_spark_session(app_name="f_load_subway_passengers",notebook_name="test", notebook_namespace="demo01-kf", \
            executor_instances="1", executor_memory="1G", executor_cores="1", driver_memory="1G", driver_cores="1" )
em = EltManager(spark, config_file="config.yaml", domain="life", record_type="mart", record_prefix="hopt")

:: loading settings :: url = jar:file:/usr/local/spark-3.5.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-872a2ada-1ac4-4c16-a4e8-b5709498ddf7;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-common;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-protobuf_3_7;1.1.1 in central
	found org.apache.hadoop#hadoop-annotations;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found com.google.guava#guava;27.0-jre in central
	found com.google.guava#failureaccess;1.0 in central
	found com.google.guava#listenablefuture;9999.0-empty-to-avoid-conflict-with-guava in central
	found com.google.code.findbugs#jsr305;3.0.2 i

In [8]:
# input parameters
WK_YM = '202505'

### JSON > Bronze

In [9]:
# PERMISSIVE 모드를 사용하여 손상된 레코드를 처리하고, 인코딩을 명시합니다.
# 파일에 손상된 줄이 있을 경우, 해당 줄은 '_corrupt_record'라는 새 컬럼에 저장됩니다.
df = spark.read.option("header","true").csv("s3a://paasup/datasource/life/CARD_SUBWAY_MONTH_202505.csv")


df.printSchema()

root
 |-- 사용일자: string (nullable = true)
 |-- 노선명: string (nullable = true)
 |-- 역명: string (nullable = true)
 |-- 승차총승객수: string (nullable = true)
 |-- 하차총승객수: string (nullable = true)
 |-- 등록일자: string (nullable = true)



In [10]:
source_type = "csv1";  source_topic = "paasup"; source_dpath="datasource/life"
target_type = "delta1";   target_topic = "paasup"; target_dpath="bronze/life"

em.init_rsm(source_type, source_topic, source_dpath, target_type, target_topic, target_dpath, 500000)

In [11]:
source_objects = ["CARD_SUBWAY_MONTH_{WK_YM}.csv".format(WK_YM=WK_YM)]; target_object = "CARD_SUBWAY_MONTH"
#source_tm = em.getSourceManager()
#source_df = source_tm.loadTable(source_objects[0])

source_inc_query = f"""
select * from CARD_SUBWAY_MONTH
""" 
# target_condition = toTargetCondition(source_df, key)
target_condition = f" `사용일자` like '{WK_YM}'||'%' "

#source_df, target_df, valid = em.ingest_full(source_objects, target_object)

source_df, target_df, valid = em.ingest_increment(source_objects, target_object, source_inc_query, target_condition, delemeter=",")

2025-07-01 03:09:46,797 - INFO - EltManager - ETL/IC Started : [ delta1 paasup bronze/life CARD_SUBWAY_MONTH (['CARD_SUBWAY_MONTH_202505.csv']) ]
+--------+------+----+------------+------------+--------+
|사용일자|노선명|역명|승차총승객수|하차총승객수|등록일자|
+--------+------+----+------------+------------+--------+
|20250501|분당선|야탑|       17571|       18765|20250504|
|20250501| 8호선|석촌|        6970|        8042|20250504|
|20250501|분당선|서현|       16548|       17173|20250504|
|20250501|경부선|독산|        8851|        8507|20250504|
|20250501|경부선|수원|       35103|       38426|20250504|
+--------+------+----+------------+------------+--------+
only showing top 5 rows



25/07/01 03:09:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/07/01 03:10:12 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.
25/07/01 03:10:13 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.


2025-07-01 03:10:18,283 - INFO - EltManager - Source Loading : csv1 paasup datasource/life ['CARD_SUBWAY_MONTH_202505.csv'] / source_size=19,139 / elipsed=31.49
2025-07-01 03:10:18,284 - INFO - EltManager - Target  Saving : delta1 paasup bronze/life CARD_SUBWAY_MONTH / delsert_size=19,139 (before=75,905, after=75,905, del=19,139) / elipsed=0.00


2025-07-01 03:10:19,936 - INFO - EltManager - ETL/IC Done : [ delta1 paasup bronze/life CARD_SUBWAY_MONTH / True (19,139, 19,139, 0) / 33.14 ]


### Bronze > Gold

In [12]:
source_type = "delta1"; source_topic = "paasup";  source_dpath="bronze/life"
target_type = "delta1"; target_topic = "paasup";  target_dpath="gold"
em.init_rsm(source_type, source_topic, source_dpath, target_type, target_topic, target_dpath, 50000000)

In [13]:
source_objects = ["CARD_SUBWAY_MONTH"];  target_object = "subway_passengers"

source_inc_query = f"""
                        select  `사용일자` use_date
                           , `노선명` line_no
                           , `역명` station_name
                           , `승차총승객수` pass_in
                           , `하차총승객수` pass_out
                           , `등록일자` reg_date
                        from CARD_SUBWAY_MONTH
                        where `사용일자` like '{WK_YM}'||'%'
"""
target_condition = f"""
 use_date like '{WK_YM}'||'%'
"""

source_df, target_df, valid = em.ingest_increment(source_objects, target_object, source_inc_query, target_condition)

2025-07-01 03:10:23,298 - INFO - EltManager - ETL/IC Started : [ delta1 paasup gold subway_passengers (['CARD_SUBWAY_MONTH']) ]


25/07/01 03:10:31 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.
25/07/01 03:10:33 WARN AmazonHttpClient: SSL Certificate checking for endpoints has been explicitly disabled.


2025-07-01 03:10:38,111 - INFO - EltManager - Source Loading : delta1 paasup bronze/life ['CARD_SUBWAY_MONTH'] / source_size=19,139 / elipsed=14.81
2025-07-01 03:10:38,112 - INFO - EltManager - Target  Saving : delta1 paasup gold subway_passengers / delsert_size=19,139 (before=75,905, after=75,905, del=19,139) / elipsed=0.00


2025-07-01 03:10:39,441 - INFO - EltManager - ETL/IC Done : [ delta1 paasup gold subway_passengers / True (19,139, 19,139, 0) / 16.14 ]


### Gold > MART

In [14]:
source_type = "delta1"; source_topic = "paasup";  source_dpath="gold"
target_type = "mart"; target_topic = "postgres";  target_dpath=None
em.init_rsm(source_type, source_topic, source_dpath, target_type, target_topic, target_dpath, 50000000)

In [15]:
source_objects = ["subway_passengers"];  target_object = "subway_passengers"
#source_tm = em.getSourceManager()
#source_df = source_tm.loadTable(source_objects[0])
#source_df.show()

# source_inc_query = f""" select '20250404' use_date
#                               , 'bbb' line_no
#                               , 'bbb' station_name
#                               , 100 pass_in
#                               , 100 pass_out
#                               , '20250505' reg_date
#                     """ 

source_inc_query = f"""
                        select  use_date
                              , line_no
                              , station_name
                              , cast(pass_in as int) pass_in
                              , cast(pass_out as int) pass_out
                              , reg_date
                        from subway_passengers
                        where use_date like '{WK_YM}'||'%'
"""
target_condition = f"""
 use_date like '{WK_YM}'||'%' 
"""

source_df, target_df, valid = em.ingest_increment(source_objects, target_object, source_inc_query, target_condition)

2025-07-01 03:10:40,066 - INFO - EltManager - ETL/IC Started : [ mart postgres _ subway_passengers (['subway_passengers']) ]


2025-07-01 03:10:51,696 - INFO - EltManager - Source Loading : delta1 paasup gold ['subway_passengers'] / source_size=19,139 / elipsed=11.63
2025-07-01 03:10:51,697 - INFO - EltManager - Target  Saving : mart postgres _ subway_passengers / delsert_size=19,139 (before=2,236,524, after=2,236,524, del=19,139) / elipsed=0.00


2025-07-01 03:10:52,554 - INFO - EltManager - ETL/IC Done : [ mart postgres _ subway_passengers / True (19,139, 19,139, 0) / 12.49 ]


In [16]:
spark.stop()

25/07/01 03:12:08 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed.
